In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.models import Model
import numpy as np
import h5py
import random

Using TensorFlow backend.


In [3]:
f = h5py.File("activity_frames.hdf5")
num_frames_per_sample = 10

In [4]:
def multimodal_generator(file, batch_size):
    while True:
        total_size = file['x'].shape[0]
        index = random.sample(range(0, total_size), batch_size)
        x = file['x'][sorted(index)]
        y = file['y'][sorted(index)]
        x = x.reshape(x.shape[0]*x.shape[1], x.shape[2], x.shape[3], x.shape[4])
        y = np.eye(20)[np.repeat(y, num_frames_per_sample)]
        x = x.astype("float") / 255.0
        yield x, y

In [5]:
input_tensor = Input(shape=(224, 224, 3))
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [6]:
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
for layer in base_model.layers:
    layer.trainable = False

In [8]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
model.fit_generator(multimodal_generator(f, batch_size=90), steps_per_epoch=100, epochs=5, verbose=1)

Epoch 1/5
